In [ ]:
import os

import mokapot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from wispy import theme

sns.set()
pal = theme.paper()
TWO_COL = 180 / 25.4
HEIGHT = 3.5
ONE_COL = 88 / 25.4

## Load the results

In [ ]:
os.makedirs("figures", exist_ok=True)
res_dir = "combined-out"
res = {}
for level in ["psms", "peptides", "proteins"]:
    res[level] = pd.read_csv(os.path.join(res_dir, f"{level}.txt"), sep="\t")

## A helper function to make the figure

In [ ]:
def comparison_plot(x, y, ax=None):
    """
    Create a scatter plot with equal axis scales.

    Plot x against y, where the axis limits are scaled
    to be equal. Additionally a y=x line is added.

    Parameters
    ----------
    x, y : numpy.ndarray
        The points to plot
    ax : matplotlib.axes.Axes
        The matplotlib axes to plot on.

    Returns
    -------
    matplotlib.axes.Axes
        The axes with the plot.
    """
    if ax is None:
        ax = plt.gca()
        
    corr = np.corrcoef(x, y)[0, 1]
    ax.text(0.05, 0.95, f"Pearson's r\n{corr:0.4f}",
            transform=ax.transAxes, va="top", fontsize="small")
    
    #cbar = ax.hexbin(x, y, mincnt=1, gridsize=100, bins="log")
    ax.scatter(x, y, s=1, alpha=0.5, edgecolor="none")
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]),
            np.max([ax.get_xlim(), ax.get_ylim()])]
    
    ax.plot(lims, lims, c="black", zorder=0, linewidth=1)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    return ax

## Make a giant figure

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(TWO_COL, 6))
labels = {"psms": "PSMs", "peptides": "Peptides", "proteins": "Proteins"}

for col, level in zip(axs.T, ["psms", "peptides", "proteins"]):
    df = res[level]
    
    # q-value curves
    mokapot.plot_qvalues(df["percolator q-value"], ax=col[0], 
                         label="Percolator",threshold=0.1)
    mokapot.plot_qvalues(df["mokapot q-value"], ax=col[0], label="mokapot", 
                         linestyle="dashed", threshold=0.1)
    col[0].legend(fontsize="small")
    col[0].set_ylabel(f"Accepted {labels[level]}")
    
    # q-values
    comparison_plot(
        df["percolator q-value"], df["mokapot q-value"], ax=col[1]
    )
    col[1].set_xlabel("Percolator q-value")
    col[1].set_ylabel("mokapot q-value")
    col[1].annotate(labels[level], (0.95, 0.05),
                    xycoords="axes fraction",
                    va="bottom", ha="right", 
                    fontsize="small")
    
    # PEPs
    comparison_plot(
        df["percolator PEP"], df["mokapot PEP"], ax=col[2]
    )
    col[2].set_xlabel("Percolator PEP")
    col[2].set_ylabel("mokapot PEP")
    col[2].annotate(labels[level], (0.95, 0.05),
                    xycoords="axes fraction",
                    va="bottom", ha="right", 
                    fontsize="small")


for ax, label in zip(axs.flatten(), "abcdefghi"):
    lab_y = 105
    ax.annotate(
        label, 
        (-10, lab_y), 
        xycoords="axes points", 
        fontweight='bold', 
        va='top', 
        ha='right'
    )
    
    
plt.tight_layout()
plt.savefig("figures/perc_comp.png", dpi=300)